# daschlab demo: RY Cnc

RY Cnc is a nice eclipsing binary with a period of 1.092943 days. Let's explore DASCH data for this system using the **daschlab** toolkit!

---

## **Important:** Scripted tutorial

Although this notebook contains comments and descriptive text, *daschlab* is **interactive** software and it's very difficult to express some of the key steps in writing.

#### **New users should follow [this tutorial slideshow][slides]** while progressing through this notebook. Open it in a new window now if it isn't already!

[slides]: https://dasch.cfa.harvard.edu/dr7/rycnc/

---

## Preliminaries

In [ ]:
# Get the main module:
import daschlab

# Set up Bokeh plots:
from bokeh.io import output_notebook
output_notebook()

# Get some other imports that will be useful:
from astropy import units as u
from bokeh.plotting import figure, show
import numpy as np
from daschlab.photometry import AFlags, BFlags

---

## Set up the session

Here we configure our analysis session: where the data are stored, what target we're interested in, and which photometric reference catalog to use.

In [ ]:
# Create or reload our session; data files stored in a directory with a
# name derived from that of our target source.
sess = daschlab.open_session(source="V* RY Cnc")

In [ ]:
# Set (or confirm) the target position, from a SIMBAD query:
sess.select_target(name="V* RY Cnc")

In [ ]:
# Specify that we're using the APASS photometric reference catalog.
# The alternative is ATLAS, which has better astrometry but whose photometric calibration
# introduces artifacts in the long-term lightcurves. It should only be used if you *only*
# care about astrometry
sess.select_refcat("apass")

---

## Display the "refcat" sources

We can use daschlab's WWT integration to display the catalog sources of interest on the sky.

#### **Before proceeding, make sure that the WWT JupyterLab app is open** — the [companion slideshow][slides] shows what to do

Once that's done, we can proceed:

[slides]: https://dasch.cfa.harvard.edu/dr7/rycnc/

In [ ]:
await sess.connect_to_wwt()

The `sess.refcat()` method returns an Astropy table of catalog information, sorted by proximity to the session query target.

In [ ]:
# Dump info about the refcat sources nearest our target:
sess.refcat()[:8]

In [ ]:
# Display all downloaded data in WWT
sess.refcat().show()

You might notice an astrometric offset between the catalog and basemap imagery. It's an unfortunate feature of the default WWT DSS mosaic. You can fix it by changing the WWT background to **PanSTARRS1 3pi**.

In [ ]:
# The result of this command will change depending on what (if anything) you 
# last clicked on in the WWT viewer.
sess.wwt().most_recent_source

---

## Display a nice sample cutout

A good way to start an analysis is to pull up a high-quality DASCH cutout around our target.

We'll identify such a cutout by finding one that detects our source of interest and has a deep "local limiting magnitude". In order to get this limiting magnitude information, we download the lightcurve data for our target.

In [ ]:
# the refcat sources are sorted by proximity to our search location
# and assigned "local ID" numbers sequentially. So, target #0 is
# almost always the one that you're interested in.
TARGET_ID = 0

lc = sess.lightcurve(TARGET_ID)
# We'll explore the lightcurve object in more detail soon.
# Also, this manual analysis is superseded by `sess.exposures().candidate_nice_cutouts()`

Make a temporary table to sort by the limiting magnitude, and print out a few rows to look for what might be a good plate to show:

In [ ]:
# Make a temporary table to sort by the limiting magnitude. Print out a 
# few rows to look for what might be a good exposure to show:
tmp_dets = lc.keep_only.nonrej_detected()
tmp_dets.sort(["limiting_mag_local"], reverse=True)
cols = "limiting_mag_local exp_local_id series platenum fwhm_world ellipticity background".split()
tmp_dets[:6][cols]

In [ ]:
# This exp_local_id looks good ...
sess.exposures().show(10721)

---

## Check out the raw lightcurve

In [ ]:
lc = sess.lightcurve(TARGET_ID)
lc.summary()

In [ ]:
# Create a Bokeh interactive plot
lc.plot()

In [ ]:
# Check out the full set of columns available
lc[:8]

---

## Plot phased with the known period

In [ ]:
PERIOD = 1.092943 # days

In [ ]:
# the offset here gets the eclipse to land at ph ~ 0.5:
lc['ph'] = (lc['time'].jd - 0.4) % PERIOD

In [ ]:
lc.plot(x_axis="ph")

---

## Filter data: standard AFLAGS

In [ ]:
STANDARD_BAD_AFLAGS = (
    AFlags.HIGH_BACKGROUND | 
    AFlags.LARGE_ISO_RMS | 
    AFlags.LARGE_LOCAL_SMOOTH_RMS | 
    AFlags.CLOSE_TO_LIMITING | 
    AFlags.BIN_DRAD_UNKNOWN
)

In [ ]:
lc.reject.any_aflags(STANDARD_BAD_AFLAGS, tag="aflags")

In [ ]:
lc.plot(x_axis="ph")

---

## Filter data: astrometric offset

In [ ]:
lc["sep"] = sess.query().pos_as_skycoord().separation(lc["pos"]).arcsec

In [ ]:
# Include already-rejected points to make the trend clear:
lc.scatter("sep", "magcal_magdep", rejects=True)

Hm, there are some points with large offsets and they definitely seem to have issues!

In [ ]:
# To understand the dubious points better, look at the underlying imagery associated with one of them:
sess.exposures().show(4104)

In [ ]:
# Reject detections whose fitted positions are farther than 15 arcsec from the true source location
lc.reject.sep_above(15 * u.arcsec, tag="sep")

In [ ]:
lc.plot(x_axis="ph")

---

## Filter data: low time accuracy

In [ ]:
# It turns out that the remaining low point has poor "time accuracy" -- 
# this column is a timestamp uncertainty in days
lc[3453]["time_accuracy"]

Note that this filter is only important since this system has a relatively short orbital period (~1 day). For other science applications, the time accuracy might not matter.

In [ ]:
# Nondetections have "masked" values for this column (although in principle they
# don't have to), so we must fill in false values for those - the lack of data
# doesn't mean that those rows are invalid.
poor_time_acc = (lc["time_accuracy"] > 0.2 * u.d).filled(False)

# How many columns have high phase uncertainty?
poor_time_acc.sum()

In [ ]:
lc.reject.where(poor_time_acc, tag="timeacc")

In [ ]:
lc.plot(x_axis="ph")

---

## Filter data: suspected defects

Let's examine the last remaining obvious outlier.

In [ ]:
sess.exposures().show(5337)

In [ ]:
lc[1346].flags()

In [ ]:
lc.reject.any_aflags(AFlags.SUSPECTED_DEFECT, tag="aflags")

In [ ]:
lc.plot(x_axis="ph")

That's the end of this notebook ... but we've only dealt with the obvious outlier detections in this lightcurve, and done so in a very coarse way! That's because this is an introductory tutorial. Consult [the DASCH documentation](https://dasch.cfa.harvard.edu/dr7/) for guides on more sophisticated analysis techniques.